<a href="https://colab.research.google.com/github/IseLei18-code/NBA-picks/blob/main/Sihaya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This following data analysis project aims to use past and recent data from NBA players and determine not only the bets on prizebets, more or less, and also the probability of us winning.

# EDA And Data Cleaning


In [4]:
%pip install -q nba_api
import pandas as pd
pd.set_option('display.max_columns', None)
import re
import numpy as np
import time
import requests
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import LeagueDashPlayerStats

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 8.6 MB/s eta 0:00:00


In [5]:
test_url = 'https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=PerGame&Scope=S&Season=2025-26&SeasonType=Regular%20Season&StatCategory=PTS'
test_perMin_last = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=10&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerMinute&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2025-26&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

In [6]:
# manipulate api request to get datas from NBA Stats Leaders

ldr = LeagueDashPlayerStats(
    season='2025-26',
    season_type_all_star='Regular Season',
    measure_type_detailed_defense='Base',
    per_mode_detailed='PerGame',
    last_n_games=10
)
df = ldr.get_data_frames()[0]
df.head()

#curry_rows = df[df['PLAYER_NAME'] == 'Stephen Curry']
#print(float(curry_rows['PTS'].iloc[0]))

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,TEAM_COUNT
0,1631260,AJ Green,AJ,1610612749,MIL,26.0,10,5,5,0.500,27.8,3.3,7.1,0.465,2.9,6.5,0.446,0.8,1.2,0.667,0.1,2.6,2.7,2.1,0.9,0.4,0.1,0.1,3.7,1.5,10.3,1.7,17.3,0,0,19.0,1,128,97,221,117,193,195,219,29,45,76,263,249,289,400,189,251,176,240,284,311,326,12,190,169,152,224,107,12,193,1
1,1642358,AJ Johnson,AJ,1610612764,WAS,20.0,6,0,6,0.000,5.5,0.3,1.7,0.200,0.0,0.3,0.000,0.5,0.7,0.750,0.3,0.2,0.5,0.5,0.2,0.0,0.0,0.7,0.5,0.5,1.2,-2.5,2.4,0,0,2.2,298,404,81,404,411,429,396,433,362,386,362,300,322,216,326,426,423,363,397,385,334,89,403,339,425,302,438,107,12,438,1
2,203932,Aaron Gordon,Aaron,1610612743,DEN,30.0,9,8,1,0.889,28.7,5.2,11.0,0.475,1.4,4.4,0.325,4.3,4.9,0.886,1.4,3.8,5.2,1.6,1.6,0.9,0.0,0.3,1.4,3.2,16.2,12.3,25.9,0,0,26.2,177,17,349,43,107,95,103,201,141,119,218,42,48,87,99,101,98,229,131,143,334,204,286,78,79,10,125,107,12,115,1
3,1628988,Aaron Holiday,Aaron,1610612745,HOU,29.0,5,5,0,1.000,6.0,0.8,1.6,0.500,0.4,1.0,0.400,0.6,0.6,1.000,0.0,0.6,0.6,1.2,0.8,0.0,0.2,0.0,0.6,0.6,2.6,0.0,4.9,0,0,5.2,334,128,431,1,404,385,401,130,294,331,109,292,330,1,404,395,417,270,258,385,238,348,398,331,363,222,391,107,12,381,1
4,1630174,Aaron Nesmith,Aaron,1610612754,IND,26.0,9,1,8,0.111,32.0,5.4,14.6,0.374,3.2,8.6,0.377,3.1,4.0,0.778,1.9,2.4,4.3,1.8,0.8,0.9,0.3,1.0,2.1,3.4,17.2,-3.9,28.0,0,0,29.0,177,339,19,388,54,93,49,354,14,14,145,72,70,193,66,206,141,203,270,143,168,26,161,66,70,350,104,107,12,88,1


# NLP (Regex for now)
Take in user query, parse, and extract key terms using Regex.

Sets variables:
*   player_name (Lebron James)
*   stats (pts, ast, reb)
*   over (True) and under (False)
*   value (25)

User query: Stephen Curry over 25 points
Variables:
*   player_name = 'Stephen Curry'
*   stats = 'pts'
*   over = True
*   value = 25


In [49]:
STAT_MAP = {
    "pt": "pts", "pts": "pts", "point": "pts", "points": "pts",
    "reb": "reb", "rebs": "reb", "rebound": "reb", "rebounds": "reb",
    "ast": "ast", "asts": "ast", "assist": "ast", "assists": "ast",
    "blk": "blk", "blks": "blk", "block": "blk", "blocks": "blk",
    "stl": "stl", "stls": "stl", "steal": "stl", "steals": "stl"
}

pattern = re.compile(
    r"""
    ^\s*
    (?P<player>[\w\.\-\'\s]+?)      # player name
    \s+
    (?P<dir>over|under)             # over / under
    \s+
    (?P<value>\d+(?:\.\d+)?)        # number like 25 or 8.5
    \s+
    (?P<stat>
        points?|pts?|pt|           # points
        rebounds?|rebs?|reb|       # rebounds
        assists?|asts?|ast|        # assists
        blocks?|blks?|blk|         # blocks
        steals?|stls?|stl          # steals
    )
    \s*$
    """,
    re.IGNORECASE | re.VERBOSE,
)

def parse_user_query(query: str) -> dict:
    m = pattern.match(query)
    if not m:
        raise ValueError(f"Could not parse query: {query}")

    player_name = m.group("player").strip()
    direction = m.group("dir").lower()
    over = direction == "over"
    raw_value = m.group("value")
    value = float(raw_value) if "." in raw_value else int(raw_value)

    stat_word = m.group("stat").lower()
    # normalize to pts / ast / reb /
    key = next(k for k in STAT_MAP if stat_word.startswith(k))
    stats = STAT_MAP[key]

    return {
        "player_name": player_name,
        "stats": stats,
        "over": over,
        "value": value,
    }

# Example:
user_query = input("Enter Query:")
vars_parsed = parse_user_query(user_query)
player_name = vars_parsed["player_name"]
print(vars_parsed)


Enter Query:Deni Avdija over 23.5 pts
{'player_name': 'Deni Avdija', 'stats': 'pts', 'over': True, 'value': 23.5}


# Models
3 models will be used

## Model 1 (Elastic Net Linear Regression):
Focuses on the player's most recent games played (3, 5, 7, 10) and considers the other metrics that may affect the player's behavior.

Change LastNGames=3, 5, 7, 10
Change PerMode = PerMinute

### Data Cleaning & Feature Engineering

In [50]:
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

ldr_last10 = LeagueDashPlayerStats(
    season='2025-26',
    season_type_all_star='Regular Season',
    measure_type_detailed_defense='Base',
    per_mode_detailed='PerGame',
    last_n_games=10
)

ldr_last5 = LeagueDashPlayerStats(
    season='2025-26',
    season_type_all_star='Regular Season',
    measure_type_detailed_defense='Base',
    per_mode_detailed='PerGame',
    last_n_games=5
)

ldr_last3 = LeagueDashPlayerStats(
    season='2025-26',
    season_type_all_star='Regular Season',
    measure_type_detailed_defense='Base',
    per_mode_detailed='PerGame',
    last_n_games=3
)

# transfer tables above into player rows
df_10 = ldr_last10.get_data_frames()[0]
df_5  = ldr_last5.get_data_frames()[0]
df_3  = ldr_last3.get_data_frames()[0]

# columns we care about
cols_keep_pts = [
    'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION',
    'PTS', 'MIN', 'FGA', 'FTA', 'FGM', 'FG_PCT'
]

df_10 = df_10[cols_keep_pts].copy()
df_5  = df_5[cols_keep_pts].copy()
df_3  = df_3[cols_keep_pts].copy()

def rename_with_suffix(df, suffix):
    rename_map = {
        'PTS': f'PTS_{suffix}',
        'MIN': f'MIN_{suffix}',
        'FGA': f'FGA_{suffix}',
        'FTA': f'FTA_{suffix}',
        'FGM': f'FGM_{suffix}',
        'FG_PCT': f'FG_PCT_{suffix}'
    }
    return df.rename(columns=rename_map)

df_10 = rename_with_suffix(df_10, 'last10')
df_5  = rename_with_suffix(df_5,  'last5')
df_3  = rename_with_suffix(df_3,  'last3')

# merge the 3/5/10 windows on PLAYER_ID
features_current = df_10.merge(
    df_5[['PLAYER_ID', 'PTS_last5', 'MIN_last5', 'FGA_last5', 'FTA_last5', 'FGM_last5', 'FG_PCT_last5']],
    on='PLAYER_ID',
    how='inner'
).merge(
    df_3[['PLAYER_ID', 'PTS_last3', 'MIN_last3', 'FGA_last3', 'FTA_last3', 'FGM_last3', 'FG_PCT_last3']],
    on='PLAYER_ID',
    how='inner'
)

# trend features
features_current['PTS_trend_3_10'] = features_current['PTS_last3'] - features_current['PTS_last10']
features_current['MIN_trend_3_10'] = features_current['MIN_last3'] - features_current['MIN_last10']
features_current['FGA_trend_3_10'] = features_current['FGA_last3'] - features_current['FGA_last10']

# feature columns for the model
feature_cols = [
    'PTS_last3', 'PTS_last5',
    'MIN_last3', 'MIN_last5',
    'FGA_last3', 'FGA_last5',
    'FTA_last3', 'FTA_last5',
    'FG_PCT_last3', 'FG_PCT_last5',
    'PTS_trend_3_10', 'MIN_trend_3_10', 'FGA_trend_3_10'
]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

model = Pipeline([
    ('scaler', StandardScaler()),
    ('enet', ElasticNetCV(
        l1_ratio=[0.1, 0.5, 0.9],
        alphas=np.logspace(-3, 1, 20),
        cv=5,
        n_jobs=-1
    ))
])

# fit only on TRAIN
model.fit(X_train, y_train)

row = features_current[features_current['PLAYER_NAME'] == player_name]

if row.empty:
    print(f"No stats found for player: {player_name}")
else:
    X_player = row[feature_cols].fillna(0)
    y_pred_player = model.predict(X_player)[0]  # scalar

    print(f"Predicted points for next game, {player_name}: {y_pred_player:.2f}")

# evaluate on TEST (unseen during training)
y_pred_test = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_test)
rmse = mean_squared_error(y_test, y_pred_test)

print("Test MAE:", mae)
print("Test RMSE:", rmse)

Predicted points for next game, Deni Avdija: 26.66
Test MAE: 0.0009371139032520849
Test RMSE: 1.8645977762754853e-06
